In [2]:
import pandas as pd
import numpy as np
import warnings
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras import callbacks
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from keras.layers import Dense, Dropout
from sklearn.model_selection import KFold
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping

warnings.filterwarnings(action='ignore')
tf.config.set_visible_devices([], 'GPU')

In [4]:
df = pd.read_csv("data/modified_train_0417.csv")
x_train = df.drop(columns=['loan_status', 'addr_state', 'loan_amnt_intallment_rate'])
y_train = df['loan_status']

In [5]:
nan_indices = np.isnan(x_train).any(axis=1)
x_train = x_train[~nan_indices]
y_train = y_train[~nan_indices]

In [6]:
x_train.shape

(1091767, 93)

In [7]:
# minmax scaler 찾아보기
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x_train)

# PCA로 차원 축소
pca = PCA(n_components=90)
X_pca = pca.fit_transform(X_scaled)
x_train = X_pca.astype('float32')
# stratify로 학습에 용이하게 비율 유지
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)

In [8]:
y_train=to_categorical(y_train, 2).astype(int)
y_validation=to_categorical(y_validation, 2).astype(int)
# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder()
# y_train = y_train.values
# y_train = encoder.fit_transform(y_train.reshape(-1, 1))

In [9]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())    
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [42]:
def build_model(optimizer):
    # Define and compile the model
    model = keras.Sequential([
        Dense(45, input_dim=90, activation='relu'),
        Dropout(0.2),
        Dense(45, activation='relu'),
        Dropout(0.2),
        Dense(45, activation='relu'),
        Dropout(0.2),
        Dense(10),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model
skfold = KFold(random_state=30,
           n_splits=5,
           shuffle=True
          )
model = KerasClassifier(build_fn = build_model)
parameters = {'batch_size': [1000,
                             5000,
                             10000],
              'epochs': [20,
                         45,
                         60,
                         100],
              'optimizer': ['adam', 
                            'SGD'
                            ]}
random_search = RandomizedSearchCV(estimator = model,
                           param_grid = parameters,
                           cv = skfold)
# early_stopping = EarlyStopping(monitor='val_loss',min_delta=0.01)
random_search.fit(x_train, y_train)
print("최고의 파라미터 :", random_search.best_params_)
print("최고 평균 정확도 : {}".format(random_search.best_score_))

Epoch 1/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4791 - accuracy: 0.7927
Epoch 2/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4616 - accuracy: 0.7980
Epoch 3/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4589 - accuracy: 0.7986
Epoch 4/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4578 - accuracy: 0.7992
Epoch 5/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4570 - accuracy: 0.7993
Epoch 6/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4563 - accuracy: 0.7994
Epoch 7/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4559 - accuracy: 0.7997
Epoch 8/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4557 - accuracy: 0.7997
Epoch 9/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4550 - accuracy: 0.8000
Epoch 10/20
699/699 [==============================] - 2s 3ms/step - loss: 0.4547 - accuracy: 0.8000

노드 및 레이어 갯수 확인하기

In [38]:
def build_model(num_layers, num_nodes):
    # Define and compile the model
    model = keras.Sequential()
    model.add(Dense(num_nodes, input_dim=90, activation='relu'))
    for _ in range(num_layers):
        model.add(Dense(num_nodes, activation='relu'))
        model.add(Dropout(0.2))
    model.add(Dense(10))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=["accuracy"])
    return model
kfold = KFold(random_state=30,
           n_splits=2,
           shuffle=True
          )
model = KerasClassifier(build_fn = build_model, epochs=10, batch_size=100, verbose=1)
parameters = {'num_layers': [2, 3],
              'num_nodes': [45]
              }
random_search = RandomizedSearchCV(estimator = model,
                           param_grid = parameters,
                           cv = kfold)
random_search.fit(x_train, y_train)
print(f"최고의 파라미터: {random_search.best_params_}")
print(f"최고 평균 정확도: {random_search.best_score_}")

Epoch 1/10
4368/4368 [==============================] - 5s 968us/step - loss: 0.4640 - accuracy: 0.7981
Epoch 2/10
4368/4368 [==============================] - 4s 953us/step - loss: 0.4567 - accuracy: 0.7997
Epoch 3/10
4368/4368 [==============================] - 4s 956us/step - loss: 0.4553 - accuracy: 0.7999
Epoch 4/10
4368/4368 [==============================] - 4s 879us/step - loss: 0.4540 - accuracy: 0.8007
Epoch 5/10
4368/4368 [==============================] - 4s 868us/step - loss: 0.4530 - accuracy: 0.8008
Epoch 6/10
4368/4368 [==============================] - 4s 879us/step - loss: 0.4524 - accuracy: 0.8010
Epoch 7/10
4368/4368 [==============================] - 4s 866us/step - loss: 0.4517 - accuracy: 0.8014
Epoch 8/10
4368/4368 [==============================] - 4s 878us/step - loss: 0.4511 - accuracy: 0.8016
Epoch 9/10
4368/4368 [==============================] - 4s 870us/step - loss: 0.4509 - accuracy: 0.8015
Epoch 10/10
4368/4368 [==============================] - 2s 424u

In [40]:
result = random_search.cv_results_
result

{'mean_fit_time': array([39.46640146, 46.33870709]),
 'std_fit_time': array([0.49524152, 0.34758794]),
 'mean_score_time': array([1.99926245, 2.04459751]),
 'std_score_time': array([0.04201853, 0.02798259]),
 'param_num_layers': masked_array(data=[2, 3],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_num_nodes': masked_array(data=[45, 45],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'num_layers': 2, 'num_nodes': 45},
  {'num_layers': 3, 'num_nodes': 45}],
 'split0_test_score': array([0.7997582 , 0.79925668]),
 'split1_test_score': array([0.80125529, 0.80078131]),
 'mean_test_score': array([0.80050674, 0.800019  ]),
 'std_test_score': array([0.00074854, 0.00076231]),
 'rank_test_score': array([1, 2], dtype=int32)}

In [37]:
# 최고의 파라미터 : {'batch_size': 1000, 'epochs': 10, 'optimizer': 'adam'}
valid_accs, valid_f1s, valid_recalls, valid_precisions = [], [], [], []

# layers
model = keras.Sequential([
    Dense(45, input_dim=90, activation='relu'),
    Dropout(0.2),
    Dense(45, activation='relu'),
    Dropout(0.2),
    Dense(45, activation='relu'),
    Dropout(0.2),
    Dense(10),
    Dense(2, activation='softmax')
])
# optimizer
optimizer = Adam(learning_rate=0.01)
# compile
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy", f1_score, recall, precision])
skf = KFold(n_splits=5, shuffle=True, random_state=30)
for train_index, val_index in skf.split(x_train, y_train):
    X_train_fold, X_val_fold = x_train[train_index], x_train[val_index]
    Y_train_fold, Y_val_fold = y_train[train_index], y_train[val_index]

    # Train the model
    model.fit(X_train_fold, Y_train_fold, 
              batch_size=1000, 
              epochs=10, 
              verbose=1, 
              #callbacks=[EarlyStopping(min_delta=0.001, monitor="loss")]
              )

    # Evaluate the model on validation data
    valid_loss, valid_acc, valid_f1, valid_recall, valid_precision = model.evaluate(X_val_fold, Y_val_fold)
    valid_accs.append(valid_acc)
    valid_f1s.append(valid_f1)
    valid_recalls.append(valid_recall)
    valid_precisions.append(valid_precision)
    print("===================================")
    print("Validation accuracy:", valid_acc)
    print("Validation F1-score:", valid_f1)
    print("Validation recall:", valid_recall)
    print("Validation precision:", valid_precision)
print("###################################")
print("avg Validation accuracy:", np.mean(valid_accs))
print("avg Validation F1-score:", np.mean(valid_f1s))
print("avg Validation recall:", np.mean(valid_recalls))
print("avg Validation precision:", np.mean(valid_precisions))


Epoch 1/10
699/699 [==============================] - 2s 3ms/step - loss: 0.4662 - accuracy: 0.7967 - f1_score: 0.7967 - recall: 0.7967 - precision: 0.7967
Epoch 2/10
699/699 [==============================] - 2s 3ms/step - loss: 0.4598 - accuracy: 0.7982 - f1_score: 0.7982 - recall: 0.7982 - precision: 0.7982
Epoch 3/10
699/699 [==============================] - 2s 4ms/step - loss: 0.4585 - accuracy: 0.7986 - f1_score: 0.7986 - recall: 0.7986 - precision: 0.7986
Epoch 4/10
699/699 [==============================] - 3s 4ms/step - loss: 0.4579 - accuracy: 0.7986 - f1_score: 0.7986 - recall: 0.7986 - precision: 0.7986
Epoch 5/10
699/699 [==============================] - 2s 3ms/step - loss: 0.4576 - accuracy: 0.7985 - f1_score: 0.7985 - recall: 0.7985 - precision: 0.7985
Epoch 6/10
699/699 [==============================] - 2s 3ms/step - loss: 0.4573 - accuracy: 0.7989 - f1_score: 0.7989 - recall: 0.7989 - precision: 0.7989
Epoch 7/10
699/699 [==============================] - 2s 3ms/ste